# Heart of Helsinki using MS Azure tools

### Import needed libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
import time
from datetime import date
import datetime
from fbprophet import Prophet

### Load preprocessed history data

In [ ]:
test = pd.read_csv('serial_nunique_final.csv')
test.head()
test['ds'] = pd.to_datetime(test['date']) + pd.to_timedelta(test['hour'], unit='h')
test['y'] = test['nunique']
test.head()

### Do predictions using prophet

In [ ]:
prediction_results = pd.DataFrame(columns=['serial','timestamp','hour','nunique','type'])
for i in test['serial'].unique():
    temp = pd.DataFrame()
    filtered = test[test['serial']==i]            
    
    m = Prophet()
    m.fit(filtered);
     
    future = m.make_future_dataframe(periods=24,freq='H')              
    forecast = m.predict(future)
    
    temp['timestamp'] = forecast['ds']
    temp['serial'] = i
    temp['type'] = 'history'
    temp.iloc[-24:, temp.columns.get_loc('type')] = 'prediction'
    temp['hour'] = temp['timestamp'].dt.hour
    temp['nunique'] = forecast['yhat']
    
    prediction_results = prediction_results.append(temp)

### Save new data with forecasts included

In [ ]:
# One station has missing values, filtering forecasts that are in the past:
prediction_results = (prediction_results[~((prediction_results['timestamp'] < datetime.date(2019,8,17)) & (prediction_results['type']=='prediction'))])

prediction_results.to_csv('prediction_results.csv')

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./prediction_results.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)